In [1]:
#Load dataset
import json


with open('../data/formatted/meta_dataset.txt', 'r') as jsonFile:
    meta_augmented = json.load(jsonFile)
    
with open('../data/formatted/dataset_augmented.txt', 'r') as jsonFile:
    dataset_augmented = json.load(jsonFile)
    
with open('../data/formatted/midi_voicing_dataset.txt', 'r') as jsonFile:
    midi_voicing_dataset = json.load(jsonFile)

print(len(midi_voicing_dataset), len(dataset_augmented), len(meta_augmented))

48072 48072 48072


- Fix the Style Token
- Split the songs that are too long into two sections

In [4]:
import importlib
import formats as fmt
importlib.reload(fmt)

fmt.correctStyleTokensInMeta(meta_augmented)

all_styles = []
for song in meta_augmented:     
    all_styles.append(song['style'])

styles = set(all_styles)
print(len(styles))
print(meta_augmented[0])

for i, song in enumerate(dataset_augmented[:12]):
    print(song)

  0%|          | 0/48072 [00:00<?, ?it/s]

63
{'composer': 'George Gershwin, Ira Gershwin (1927)', 'style': 'Jazz', 'song_name': "'S Wonderful", 'tonality': 'Eb major', 'midi_key': 15, 'time_signature': '4/4', 'decade': 'Null'}
[['<style>', 0.0], ['Medium Swing', 0.0], ['|', 0.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 4.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 4.0], ['.', 4.0], ['C#', 4.0], ['o', 4.0], ['|', 4.0], ['.', 4.0], ['C#', 4.0], ['o', 4.0], ['|', 4.0], ['.', 4.0], ['D', 4.0], ['m7', 4.0], ['|', 4.0], ['.', 4.0], ['G', 4.0], ['dom7', 4.0], ['|', 4.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 4.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 0.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 4.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 4.0], ['.', 4.0], ['C#', 4.0], ['o', 4.0], ['|', 4.0], ['.', 4.0], ['C#', 4.0], ['o', 4.0], ['|', 4.0], ['.', 4.0], ['D', 4.0], ['m7', 4.0], ['|', 4.0], ['.', 4.0], ['G', 4.0], ['dom7', 4.0], ['|', 4.0], ['.', 4.0], ['C', 4.0], ['maj', 4.0], ['|', 4.0], ['.', 2.0], ['F